In [241]:
%%html
<button id='codeVisibility' style='float:right' onclick="toggleCodeVisibility()">Hide Code</button>
<script>
function toggleCodeVisibility() {
    if(window.codeVisibility==undefined) {
        window.codeVisibility=false;
    }
    if(codeVisibility) {
        $('.code_cell .input').hide();$("#codeVisibility").text('Show Code')
    } else {
        $('.code_cell .input').show();$("#codeVisibility").text('Hide Code')
    }
    codeVisibility=!codeVisibility;
}
</script>

In [242]:
import pandas as pd
from IPython.display import Javascript, display, HTML

In [243]:
%%javascript
window.numberOfCellWithName = function numberOfCellWithName(name) {
  var num=-1;
    $(".cell").map(function(i,c) {
    var k=$(c).find(".cm-comment").text();
    if(k.search(/#name=/i)==0) {
        if(k.split('=')[1]==name) {
            num=i;
        }
    }
  });
    return num;
}
window.execute=function execute(name) {
    // evaluate the expression and clear
    var num=numberOfCellWithName(name);
    if(num<0) {
        alert("There's no cell named '"+name+"'");
        return;
    }
    Jupyter.notebook.get_cell(num).execute(); // This will execute the cell with the given name
}

<IPython.core.display.Javascript object>

In [244]:
%%javascript
require.config({
    paths: {
        'd3': '//d3js.org/d3.v4.min'
    }
});

<IPython.core.display.Javascript object>

# Bidirectional d3 data visualisation in the jupyter notebook

Using the `%%html` and the `%%javascript` magic it is easy to inject interactive d3 data visualisations in jupyter notebooks. Most often, however, those visualisations are not able to send the results of the interaction back to the python code. Here, using the `Jupyter.notebook` object in javascript, we show how this bidirectional exchange of information can be achived. We use the event listeners in javascript to assign data to a python variable, and then execute a user-specified Jupyter notebook cell to demonstrate how to operate on that data.

The first example -- Chord plot -- shows the method to inject d3 into Jupyter. The second and third examples show how selections in d3 can be sent back to the python code.


# Chord plot

In [245]:
%%html
<style>
body {
  font: 10px sans-serif;
}
.group-tick line {
  stroke: #000;
}
.ribbons {
  fill-opacity: 0.5;
}
</style>
<svg width="320" height="320"></svg>

In [246]:
%%javascript
require(['d3'],function(d3) {
    var matrix = [
      [11975,  5871, 8916, 2868],
      [ 1951, 10048, 2060, 6171],
      [ 8010, 16145, 8090, 8045],
      [ 1013,   990,  940, 6907]
    ];

    var svg = d3.select("svg"),
        width = +svg.attr("width"),
        height = +svg.attr("height"),
        outerRadius = Math.min(width, height) * 0.5 - 40,
        innerRadius = outerRadius - 30;

    var formatValue = d3.formatPrefix(",.0", 1e3);

    var chord = d3.chord()
        .padAngle(0.05)
        .sortSubgroups(d3.descending);

    var arc = d3.arc()
        .innerRadius(innerRadius)
        .outerRadius(outerRadius);

    var ribbon = d3.ribbon()
        .radius(innerRadius);

    var color = d3.scaleOrdinal()
        .domain(d3.range(4))
        .range(["#000000", "#FFDD89", "#957244", "#F26223"]);

    var g = svg.append("g")
        .attr("transform", "translate(" + width / 2 + "," + height / 2 + ")")
        .datum(chord(matrix));

    var group = g.append("g")
        .attr("class", "groups")
      .selectAll("g")
      .data(function(chords) { return chords.groups; })
      .enter().append("g")
      .call(d3.drag()
        .subject(subject)
      )

    group.append("path")
        .style("fill", function(d) { return color(d.index); })
        .style("stroke", function(d) { return d3.rgb(color(d.index)).darker(); })
        .attr("d", arc);

    var groupTick = group.selectAll(".group-tick")
      .data(function(d) { return groupTicks(d, 1e3); })
      .enter().append("g")
        .attr("class", "group-tick")
        .attr("transform", function(d) { return "rotate(" + (d.angle * 180 / Math.PI - 90) + ") translate(" + outerRadius + ",0)"; });

    groupTick.append("line")
        .attr("x2", 6);

    groupTick
      .filter(function(d) { return d.value % 5e3 === 0; })
      .append("text")
        .attr("x", 8)
        .attr("dy", ".35em")
        .attr("transform", function(d) { return d.angle > Math.PI ? "rotate(180) translate(-16)" : null; })
        .style("text-anchor", function(d) { return d.angle > Math.PI ? "end" : null; })
        .text(function(d) { return formatValue(d.value); });

    g.append("g")
        .attr("class", "ribbons")
      .selectAll("path")
      .data(function(chords) { return chords; })
      .enter().append("path")
        .attr("d", ribbon)
        .style("fill", function(d) { return color(d.target.index); })
        .style("stroke", function(d) { return d3.rgb(color(d.target.index)).darker(); });

    // Returns an array of tick angles and values for a given group and step.
    function groupTicks(d, step) {
      var k = (d.endAngle - d.startAngle) / d.value;
      return d3.range(0, d.value, step).map(function(value) {
        return {value: value, angle: value * k + d.startAngle};
      });
    }
    
    function subject() {
        console.log(this);
    }
}, function (err) {
    console.log(err)
})

<IPython.core.display.Javascript object>

# Force directed clustered nodes

Click and drag on the nodes to move them around. The number of the node will be assigned to the variable `nodeNumber`

In [247]:
%%html
<svg id="force" width="320" height="320"></svg>

In [248]:
nodeNumber=0

In [262]:
#name=cluster
print(nodeNumber)

130


In [250]:
%%javascript
require(['d3'],function(d3) {
    let margin = {top: 100, right: 100, bottom: 100, left: 100};

    let width = 640,
        height = 320,
        padding = 1.5, // separation between same-color circles
        clusterPadding = 6, // separation between different-color circles
        maxRadius = 12;

    let n = 200, // total number of nodes
        m = 10, // number of distinct clusters
        z = d3.scaleOrdinal(d3.schemeCategory20),
        clusters = new Array(m);

    let svg = d3.select('#force')
        .attr('width',width)
        .append('g').attr('transform', 'translate(' + width / 2 + ',' + height / 2 + ')');

    let nodes = d3.range(200).map(() => {
        let i = Math.floor(Math.random() * m),
            radius = Math.sqrt((i + 1) / m * -Math.log(Math.random())) * maxRadius,
            d = {cluster: i, r: radius};
        if (!clusters[i] || (radius > clusters[i].r)) clusters[i] = d;
        return d;
    });

    let circles = svg.append('g')
          .datum(nodes)
        .selectAll('.circle')
          .data(d => d)
        .enter().append('circle')
          .attr('r', (d) => d.r)
          .attr('fill', (d) => z(d.cluster))
          .attr('stroke', 'black')
          .attr('stroke-width', 1)
        .call(d3.drag()
            .on("start", dragstarted)
            .on("drag", dragged)
            .on("end", dragended));
    let text = svg.append('g')
          .datum(nodes)
        .selectAll('.text')
          .data(d => d)
        .enter().append('text')
          .text( function (d,i) {return i})
          .attr("pointer-events", "none")
          .attr("font-family", "sans-serif")
          .attr("font-size", "14px");

    let simulation = d3.forceSimulation(nodes)
        .velocityDecay(0.2)
        .force("x", d3.forceX().strength(.0005))
        .force("y", d3.forceY().strength(.0005))
        .force("collide", collide)
        .force("cluster", clustering)
        .on("tick", ticked);

    function ticked() {
        circles
            .attr('cx', (d) => d.x)
            .attr('cy', (d) => d.y);
        text
            .attr('x', (d) => d.x)
            .attr('y', (d) => d.y);
    }   

    // These are implementations of the custom forces.
    function clustering(alpha) {
        nodes.forEach(function(d) {
          var cluster = clusters[d.cluster];
          if (cluster === d) return;
          var x = d.x - cluster.x,
              y = d.y - cluster.y,
              l = Math.sqrt(x * x + y * y),
              r = d.r + cluster.r;
          if (l !== r) {
            l = (l - r) / l * alpha;
            d.x -= x *= l;
            d.y -= y *= l;
            cluster.x += x;
            cluster.y += y;
          }  
        });
    }

    function collide(alpha) {
      var quadtree = d3.quadtree()
          .x((d) => d.x)
          .y((d) => d.y)
          .addAll(nodes);

      nodes.forEach(function(d) {
        var r = d.r + maxRadius + Math.max(padding, clusterPadding),
            nx1 = d.x - r,
            nx2 = d.x + r,
            ny1 = d.y - r,
            ny2 = d.y + r;
        quadtree.visit(function(quad, x1, y1, x2, y2) {

          if (quad.data && (quad.data !== d)) {
            var x = d.x - quad.data.x,
                y = d.y - quad.data.y,
                l = Math.sqrt(x * x + y * y),
                r = d.r + quad.data.r + (d.cluster === quad.data.cluster ? padding : clusterPadding);
            if (l < r) {
              l = (l - r) / l * alpha;
              d.x -= x *= l;
              d.y -= y *= l;
              quad.data.x += x;
              quad.data.y += y;
            }
          }
          return x1 > nx2 || x2 < nx1 || y1 > ny2 || y2 < ny1;
        });
      });
    }

    function dragstarted(d) {
        if (!d3.event.active) simulation.alphaTarget(0.3).restart();
        d.fx = d.x;
        d.fy = d.y;

        // find index of selected node
        var dist=-1,index;
        nodes.forEach(function(d,i) {
          var s2=Math.pow(d.x-d3.event.x,2)+Math.pow(d.y-d3.event.y,2);
          if(dist==-1 || s2<dist) {
              dist=s2;
              index=i;
          }
        });
        Jupyter.notebook.kernel.execute("nodeNumber="+index);
        execute("cluster");
    }

    function dragged(d) {
        d.fx = d3.event.x;
        d.fy = d3.event.y;
    }

    function dragended(d) {
        if (!d3.event.active) simulation.alphaTarget(0);
        d.fx = null;
        d.fy = null;
    }

}, function (err) {
    console.log(err)
})

<IPython.core.display.Javascript object>

# Hierarchical edge bundling

Mouse over the circular graph nodes to display its connections. Click on a node to assign its name to the python variable `hirarchicalNodeName`.

Code from:
https://bl.ocks.org/mbostock/7607999

Also, a version where the tension in the links can be changed interactively:
https://mbostock.github.io/d3/talk/20111116/bundle.html

The interesting part of that code is:
```
  d3.select("input[type=range]").on("change", function() {
    line.tension(this.value / 100);
    path.attr("d", function(d, i) { return line(splines[i]); });
  });
```

In [251]:
%%html
<style>
.node {
  font: 300 11px "Helvetica Neue", Helvetica, Arial, sans-serif;
  fill: #bbb;
}

.node:hover {
  fill: #000;
}

.link {
  stroke: steelblue;
  stroke-opacity: 0.4;
  fill: none;
  pointer-events: none;
}

.node:hover,
.node--source,
.node--target {
  font-weight: 700;
}

.node--source {
  fill: #2ca02c;
}

.node--target {
  fill: #d62728;
}

.link--source,
.link--target {
  stroke-opacity: 1;
  stroke-width: 2px;
}

.link--source {
  stroke: #d62728;
}

.link--target {
  stroke: #2ca02c;
}

</style>
<svg id="hierarchical" width="320" height="320"></svg>

In [252]:
hierarchicalNodeName=""

In [259]:
#name=hierarchical
print(hierarchicalNodeName)

MaxFlowMinCut


In [254]:
%%javascript
require(['d3'],function(d3) {
    var diameter = 960,
        radius = diameter / 2,
        innerRadius = radius - 120;

    var cluster = d3.cluster()
        .size([360, innerRadius]);

    var line = d3.radialLine()
        .curve(d3.curveBundle.beta(0.85))
        .radius(function(d) { return d.y; })
        .angle(function(d) { return d.x / 180 * Math.PI; });

    var svg = d3.select("#hierarchical")
        .attr("width", diameter)
        .attr("height", diameter)
      .append("g")
        .attr("transform", "translate(" + radius + "," + radius + ")");

    var link = svg.append("g").selectAll(".link"),
        node = svg.append("g").selectAll(".node");

    d3.json("flare.json", function(error, classes) {
      if (error) throw error;

      var root = packageHierarchy(classes)
          .sum(function(d) { return d.size; });

      cluster(root);

      link = link
        .data(packageImports(root.leaves()))
        .enter().append("path")
          .each(function(d) { d.source = d[0], d.target = d[d.length - 1]; })
          .attr("class", "link")
          .attr("d", line);

      node = node
        .data(root.leaves())
        .enter().append("text")
          .attr("class", "node")
          .attr("dy", "0.31em")
          .attr("transform", function(d) { return "rotate(" + (d.x - 90) + ")translate(" + (d.y + 8) + ",0)" + (d.x < 180 ? "" : "rotate(180)"); })
          .attr("text-anchor", function(d) { return d.x < 180 ? "start" : "end"; })
          .text(function(d) { return d.data.key; })
          .on("mousedown", mousedowned)
          .on("mouseover", mouseovered)
          .on("mouseout", mouseouted);
    });

    function mouseovered(d) {
      node
          .each(function(n) { n.target = n.source = false; });

      link
          .classed("link--target", function(l) { if (l.target === d) return l.source.source = true; })
          .classed("link--source", function(l) { if (l.source === d) return l.target.target = true; })
        .filter(function(l) { return l.target === d || l.source === d; })
          .raise();

      node
          .classed("node--target", function(n) { return n.target; })
          .classed("node--source", function(n) { return n.source; });
    }

    function mouseouted(d) {
      link
          .classed("link--target", false)
          .classed("link--source", false);

      node
          .classed("node--target", false)
          .classed("node--source", false);
    }

    function mousedowned(d) {
        var cmd='hierarchicalNodeName="'+d.data.key+'"';
        console.log(cmd);
        Jupyter.notebook.kernel.execute(cmd);
        execute("hierarchical");
    }

    // Lazily construct the package hierarchy from class names.
    function packageHierarchy(classes) {
      var map = {};

      function find(name, data) {
        var node = map[name], i;
        if (!node) {
          node = map[name] = data || {name: name, children: []};
          if (name.length) {
            node.parent = find(name.substring(0, i = name.lastIndexOf(".")));
            node.parent.children.push(node);
            node.key = name.substring(i + 1);
          }
        }
        return node;
      }

      classes.forEach(function(d) {
        find(d.name, d);
      });

      return d3.hierarchy(map[""]);
    }

    // Return a list of imports for the given array of nodes.
    function packageImports(nodes) {
      var map = {},
          imports = [];

      // Compute a map from name to node.
      nodes.forEach(function(d) {
        map[d.data.name] = d;
      });

      // For each import, construct a link from the source to target node.
      nodes.forEach(function(d) {
        if (d.data.imports) d.data.imports.forEach(function(i) {
          imports.push(map[d.data.name].path(map[i]));
        });
      });

      return imports;
    }
}, function (err) {
    console.log(err)
})

<IPython.core.display.Javascript object>

# Notes

A talk about the differences between d3 v3 and v4: https://iros.github.io/d3-v4-whats-new/#1